In [ ]:
import os, shutil, sys
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
#####
SAMPLE_SIZE = 100
new_names, new_model_names = [], []
sample_sessions = []
PATH_C3D_START = r"Y:\departments\research_and_development\sports_science\01_mocap_operations\hitting\v1\03_athletes"
PATH_C3D_COPY = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_hitting\data\c3d"
PATH_META_SAVE = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_hitting\data"
PATH_POI_SAVE = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_hitting\data\poi"
PATH_FULL_SIG_SAVE = r"C:\Users\kylew\Documents\GitHub\openbiomechanics\baseball_hitting\data\full_sig"
SAVE_META_CSV = True
# ###################### Connect to DB ######################
db_user_ = os.environ['CLUSTER_USERNAME_DB_BIOMECH']
db_pw_ = os.environ['CLUSTER_PASSWORD_DB_BIOMECH']
# db_dbname_ = os.environ['DATABASE_BIOMECH_PITCHING_DB']
db_dbname_ = 'biomech_hitting_db'
db_host_ = os.environ['CLUSTER_HOST_DB_BIOMECH']
db_port_ = os.environ['CLUSTER_PORT_DB_BIOMECH']
engine_string = 'mysql+mysqlconnector://'+db_user_ +':'+db_pw_+'@'+db_host_+':'+str(db_port_)+'/'+db_dbname_
engine = create_engine(engine_string,connect_args={'auth_plugin': 'mysql_native_password'}, echo=False)

#Get Data from Biomech DB
cnx = mysql.connector.connect(host=db_host_,
                                         database=db_dbname_,
                                         user=db_user_,
                                         password=db_pw_,
                                         port =db_port_,
                                         auth_plugin='mysql_native_password')

cursor = cnx.cursor()

In [ ]:
query = 'select distinct session_swing from bh_force_plate where rear_force_x is not null;'
d = pd.read_sql(query, cnx)
force_plate_swings = tuple(pd.unique(d.session_swing))

In [ ]:
query = "select u.user, s.session, poi.session_swing, s.session_mass_lbs, s.session_height_in, poi.bat_speed_mph_max_x from bh_sessions s left join bh_poi_metrics poi using(session) left join bh_users u using(user) where (s.irb=1) and (s.highest_playing_level not in ('mlb', 'other')) and (poi.session_swing in {});"
# query = "select u.user, s.session, poi.session_swing, s.session_mass_lbs, s.session_height_in, poi.bat_speed_mph_max_x from bh_sessions s left join bh_poi_metrics poi using(session) left join bh_users u using(user) where (s.irb=1) and (s.highest_playing_level not in ('other')) and (poi.session_swing in {});"
d = pd.read_sql(query.format(force_plate_swings), cnx)
users = pd.unique(d.user).tolist()
print(len(users))
# sample_users = np.random.choice(users, SAMPLE_SIZE, replace=False)
sample_users = users

In [ ]:
for user in sample_users:
# for user in users:
    user_sessions = d[d.user==user].session.tolist()
    # pick one session
    session = np.random.choice(user_sessions, 1)[0]
    sample_sessions.append(session)
sample_sessions = tuple(sample_sessions)

In [ ]:
query = "select s.*, poi.session_swing, poi.blast_bat_speed_mph_x, poi.bat_speed_mph_max_x, poi.exit_velo_mph_x, u.subject_traq from bh_sessions s left join bh_poi_metrics poi using(session) left join bh_users u using(user) where (s.session in {});"
d1 = pd.read_sql(query.format(sample_sessions), cnx)
# d1['session_date'] = pd.to_datetime(d1['session_date'])
# d1['date_of_birth'] = pd.to_datetime(d1['date_of_birth'])
# d1['age_yrs'] = d1['session_date'] - d1['date_of_birth']
# d1['age_yrs'] = np.round(d1['age_yrs'].apply(lambda x: x.days/365.25),2)
d1 = d1[['user', 'subject_traq', 'session', 'session_swing', 'session_mass_lbs', 'session_height_in', 'athlete_age', 'highest_playing_level', 'hitter_side', 'bat_weight_oz', 'bat_length_in', 'bat_speed_mph_max_x', 'blast_bat_speed_mph_x', 'exit_velo_mph_x']]

In [ ]:
query = "select u.user, s.session, pos.session_swing, pos.filename FROM raw_bh_positions pos left join bh_poi_metrics poi using(session_swing) left join bh_sessions s on(s.SESSION=pos.session) left join bh_users u ON (s.user=u.user) where (s.session in {});"
d2 = pd.read_sql(query.format(sample_sessions), cnx)
d2['user'] = d2['user'].astype(str).apply(lambda x: x.zfill(6))
d2['session'] = d2['session'].astype(str).apply(lambda x: x.zfill(6))
modelnames = d2['filename'].apply(lambda x: '_'.join(x.split('_')[0:3]) + '_model.c3d')
d2['modelnames'] = modelnames
d2['filename_stem'] = d2['filename'].apply(lambda x: '_'.join(x.split('_')[:3]))

# join d1 and d2 on session_pitch
d = d1.merge(d2, on=['session_swing'], how='left')
d.drop(columns=['session_x', 'user_x'], inplace=True)
d.rename(columns={'session_y': 'session', 'user_y':'user'}, inplace=True)
d.filename = d.filename.apply(lambda x: x.replace('.json', '.c3d'))
sample_session_swings = tuple(d.session_swing.tolist())

In [ ]:
d.head()

In [ ]:
for root, dirs, files in os.walk(PATH_C3D_START):
    for file in files:
        if file.endswith('.c3d'):
            y_drive_stem = file.split('.')[0]
            y_drive_stem = '_'.join(y_drive_stem.split('_')[0:3])
            if y_drive_stem in d['filename_stem'].values:
                uscore_count = file.count('_')
                if uscore_count <= 4: # if it's a model file
                    model_path = os.path.join(root, file)
                    model_stem = '_'.join(file.split('_')[0:3])
                    user_id = d2.loc[d2['filename_stem']==model_stem, 'user'].iloc[0]
                    session = d2.loc[d2['filename_stem']==model_stem, 'session'].iloc[0]
                    # swing = d2.loc[d2['filename_stem']==model_stem, 'session_swing'].split('_')[-1].zfill(3)
                    new_model_name = '_'.join([user_id, session, 'model.c3d'])
                    new_model_path = os.path.join(PATH_C3D_COPY, user_id, session)
                    if not os.path.exists(new_model_path):
                        os.makedirs(new_model_path)
                    new_model_path = os.path.join(new_model_path, new_model_name)
                    print('copying {} to {}'.format(file, new_model_path))
                    shutil.copyfile(model_path, new_model_path)
                elif uscore_count == 6: # if it's a swing file before handedness was added to filename
                    old_path = os.path.join(root, file)
                    height = file.split('_')[3]
                    weight = file.split('_')[4]
                    swing_num = file.split('_')[5]
                    exit_velo = file.split('_')[6]
                    file_stem = '_'.join(file.split('_')[0:3])
                    user_id = d.loc[d['filename_stem']==file_stem, 'user'].iloc[0]
                    session = d.loc[d['filename_stem']==file_stem, 'session'].iloc[0]
                    hitter_side = d.loc[d['filename_stem']==file_stem, 'hitter_side'].iloc[0]
                    new_filename = '_'.join([user_id, session, height, weight, hitter_side, swing_num, exit_velo])
                    # print('uscore = 6: ', new_filename)
                    if not os.path.exists(os.path.join(PATH_C3D_COPY, user_id, session)):
                        os.makedirs(os.path.join(PATH_C3D_COPY, user_id, session))
                    shutil.copyfile(old_path, os.path.join(PATH_C3D_COPY, user_id, session, new_filename))
                elif uscore_count == 7: # if it's a swing file after handedness was added to filename
                    old_path = os.path.join(root, file)
                    height = file.split('_')[3]
                    weight = file.split('_')[4]
                    hitter_side = file.split('_')[5]
                    swing_num = file.split('_')[6]
                    exit_velo = file.split('_')[7]
                    file_stem = '_'.join(file.split('_')[0:3])
                    user_id = d2.loc[d2['filename_stem']==file_stem, 'user'].iloc[0]
                    session = d2.loc[d2['filename_stem']==file_stem, 'session'].iloc[0]
                    new_filename = '_'.join([user_id, session, height, weight, hitter_side, swing_num, exit_velo])
                    new_filename = new_filename.replace('left', 'L').replace('right', 'R')
                    # print('uscore = 7: ', new_filename)
                    if not os.path.exists(os.path.join(PATH_C3D_COPY, user_id, session)):
                        os.makedirs(os.path.join(PATH_C3D_COPY, user_id, session))
                    shutil.copyfile(old_path, os.path.join(PATH_C3D_COPY, user_id, session, new_filename))



In [ ]:
d.head()

In [ ]:
if SAVE_META_CSV:
    d.drop(columns=['subject_traq', 'filename', 'modelnames', 'filename_stem']).to_csv(os.path.join(PATH_META_SAVE, "metadata.csv"), index=False)

In [ ]:
# plt.rcParams.update({'font.family': 'gotham'})
# fig = plt.figure(figsize=(10,12.5), facecolor='w')
# # add subplots
# ax0 = fig.add_subplot(311)
# ax1 = fig.add_subplot(312)
# ax2 = fig.add_subplot(313)

# # plot data
# ax0.hist(d.mass_lbs)
# ax0.set_xlabel('Mass (lbs)')
# ax0.set_ylabel('Count')

# ax1.hist(d.height_in)
# ax1.set_xlabel('Height (in)')
# ax1.set_ylabel('Count')

# ax2.hist(d.velo)
# ax2.set_xlabel('Pitch Speed (mph)')
# ax2.set_ylabel('Count')

In [ ]:
d = pd.read_csv(os.path.join(PATH_META_SAVE, 'metadata.csv'))
sample_session_swings = tuple(d.session_swing.tolist())

In [ ]:
# save force plate full signal data from session_pitches
query = """

SELECT 

fp.session_swing, 
fp.time, 
fp.rear_force_x, 
fp.rear_force_y, 
fp.rear_force_z,
fp.lead_force_x,
fp.lead_force_y,
fp.lead_force_z,
e.fp_force_plate_10perc_time as fp_10_time,
e.fp_force_plate_100perc_time as fp_100_time,
e.contact_time

FROM bh_force_plate fp 
LEFT JOIN bh_events e using(session_swing) 

WHERE session_swing in {};""".format(sample_session_swings)

force_plate_data = pd.read_sql(query, cnx)
force_plate_data.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'force_plate.csv'), index=False)

In [ ]:
# query = """

# SELECT 

# ef.*, 
# e.pkh_time, 
# e.fp_force_plates_time_10perc as fp_10_time,
# e.fp_force_plates_time_100perc as fp_100_time

# FROM bh_energy_flow ef 
# LEFT JOIN bh_events e using(session_pitch) 

# WHERE session_pitch in {};""".format(sample_session_swings)

# energy_flow_data = pd.read_sql(query, cnx)
# energy_flow_data.drop(columns=['session_pitch_time'], inplace=True)
# energy_flow_data.drop(columns=[x for x in energy_flow_data.columns if 'total' in x], inplace=True)
# energy_flow_data.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'energy_flow.csv'), index=False)

In [ ]:
# query = """

# SELECT 

# jm.*, 
# jf.*, 
# e.fp_force_plate_10perc_time as fp_10_time,
# e.fp_force_plate_100perc_time as fp_100_time,
# e.contact_time

# FROM bp_joint_forces jf 
# LEFT JOIN bp_joint_moments jm USING(session_pitch_time)
# LEFT JOIN bp_events e ON(jm.session_pitch=e.session_pitch)

# WHERE jm.session_pitch in {};""".format(sample_session_swings)

# forces_moments = pd.read_sql(query, cnx)
# forces_moments.drop(columns=['session_pitch_time'], inplace=True)
# forces_moments.iloc[:, 50:52] = np.nan
# forces_moments.dropna(axis=1, how='all', inplace=True)
# forces_moments.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'forces_moments.csv'), index=False)

In [ ]:
query = """

SELECT

ja.*,
e.fp_force_plate_10perc_time as fp_10_time,
e.fp_force_plate_100perc_time as fp_100_time,
e.contact_time

FROM bh_joint_angles ja 
LEFT JOIN bh_events e using(session_swing)

WHERE session_swing in {};""".format(sample_session_swings)

joint_angles = pd.read_sql(query, cnx)
joint_angles.drop(columns=['session_swing_time'], inplace=True)
joint_angles.drop(columns=[x for x in joint_angles.columns if 'virtual_lab' in x], inplace=True)
joint_angles.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'joint_angles.csv'), index=False)


In [ ]:
query = """

SELECT

jv.*,
e.fp_force_plate_10perc_time as fp_10_time,
e.fp_force_plate_100perc_time as fp_100_time,
e.contact_time

FROM bh_joint_velos jv 
LEFT JOIN bh_events e using(session_swing)

WHERE session_swing in {};""".format(sample_session_swings)

joint_velos = pd.read_sql(query, cnx)
joint_velos.drop(columns=['session_swing_time'], inplace=True)
joint_velos.drop(columns=[x for x in joint_velos.columns if 'sig_mag' in x], inplace=True)
joint_velos.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'joint_velos.csv'), index=False)

In [ ]:
query = """

SELECT

lm.*,
e.fp_force_plate_10perc_time as fp_10_time,
e.fp_force_plate_100perc_time as fp_100_time,
e.contact_time

FROM bh_landmarks lm 
LEFT JOIN bh_events e using(session_swing) 

WHERE session_swing in {};""".format(sample_session_swings)

landmarks = pd.read_sql(query, cnx)
landmarks.drop(columns=['session_swing_time'], inplace=True)
landmarks.drop(columns=[x for x in landmarks.columns if '_velo_' in x], inplace=True)
landmarks.to_csv(os.path.join(PATH_FULL_SIG_SAVE, 'landmarks.csv'), index=False)

In [ ]:
query = "select * from bh_poi_metrics where session_swing in {};".format(sample_session_swings)
poi_metrics = pd.read_sql(query, cnx)
poi_metrics.drop(columns=['height_x', 'mass_x'], inplace=True)
poi_metrics.to_csv(os.path.join(PATH_POI_SAVE, 'poi_metrics.csv'), index=False)

In [ ]:
query = """

SELECT

ht.session_swing,
ht.pitch,
ht.strike_zone,
ht.la,
ht.dist,
ht.res,
ht.bearing,
ht.vertical_distance,
ht.horizontal_distance,
ht.poi_x,
ht.poi_y,
ht.poi_z,
ht.pitch_angle

FROM bh_hittrax ht

where session_swing in {};""".format(sample_session_swings)

hittrax = pd.read_sql(query, cnx)
hittrax.to_csv(os.path.join(PATH_POI_SAVE, 'hittrax.csv'), index=False)